In [ ]:
import scipp as sc
import scippneutron as scn
from ess.diffraction.powder import calibration, conversions, normalization
import ess
import numpy as np

In [ ]:
sample = scn.load(ess.powgen.data.sample_file(),
                  advanced_geometry=True,
                  load_pulse_times=False,
                  mantid_args={'LoadMonitors': True})

In [ ]:
cal = calibration.load_calibration(
    ess.powgen.data.calibration_file(),
    mantid_args={'InstrumentFilename': 'POWGEN_Definition_2011-02-25.xml'})

In [ ]:
cal = ess.powgen.beamline.map_detector_to_spectrum(cal, detector_info=sample.coords['detector_info'].value)

In [ ]:
sample

Normalize by proton charge

In [ ]:
sample /= sample.attrs['gd_prtn_chrg'].values

In [ ]:
sample_dspacing = conversions.to_dspacing_with_calibration(sample, calibration=cal)

Vanadium

In [ ]:
def load_auxilliary_as_wavelength(filename):
    da = scn.load(filename,
                  advanced_geometry=True,
                  load_pulse_times=False,
                  mantid_args={'LoadMonitors': True})
    return da.transform_coords('wavelength',
                               graph={
                                   **scn.tof.conversions.beamline(scatter=True),
                                   **scn.tof.conversions.elastic("tof")
                               })

In [ ]:
vana = load_auxilliary_as_wavelength(ess.powgen.data.vanadium_file())

In [ ]:
empty_instr = load_auxilliary_as_wavelength(ess.powgen.data.empty_instrument_file())

In [ ]:
vana = normalization.subtract_empty_instrument(vana, empty_instr)
del empty_instr

In [ ]:
vana_dspacing = conversions.to_dspacing_with_calibration(vana, calibration=cal)

In [ ]:
vana_dspacing

In [ ]:
v = vana_dspacing.bins.concat('spectrum')
min_dspacing = sc.min(v.coords['dspacing']).value
max_dspacing = sc.max(v.coords['dspacing']).value
dspacing_edges = sc.linspace('dspacing',
                             min_dspacing,
                             max_dspacing,
                             200,
                             unit=v.coords['dspacing'].unit)
vana_histo = sc.histogram(v, bins=dspacing_edges)

In [ ]:
s = sample_dspacing.bins.concat('spectrum')
sample_corrected = s.bins / sc.lookup(vana_histo, dim='dspacing')
sample_corrected = sc.histogram(sample_corrected, bins=dspacing_edges)

In [ ]:
sample_corrected.plot()